In [1]:
!pip install -q transformers datasets accelerate faiss-cpu sentence-transformers pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 44.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 48.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

# Load and Chunk pdf

In [2]:
import fitz  # PyMuPDF

def extract_text_from_pdf_by_sentences(path):
    doc = fitz.open(path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

# Chunk pdf into paragraphs or sentences

In [3]:
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

def chunk_text(text, max_tokens=200):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2") #22M P
    
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_length = 0
    
    for sentence in sentences:
        length = len(tokenizer.tokenize(sentence))
        if current_length + length <= max_tokens:
            current_chunk += " " + sentence
            current_length += length
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_length = length
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Embed chunks with sentence transformers

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_chunks(chunks):
    return model.encode(chunks, show_progress_bar=True)

2025-06-20 22:18:57.843009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750457938.094156      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750457938.171841      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Store embeddings with FAISS

In [5]:
import faiss

def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

# RAG query

In [7]:
from transformers import pipeline
from huggingface_hub import login
import json

with open("/kaggle/input/autenti/AUTH nn.json", "r") as f:
    token_data = json.load(f)

HF_TOKEN = token_data["API_KEY"]

login(HF_TOKEN)
# 7 Billion parameters 
# English, Spanish, German, Italian, French, Portuguese, etc
# Common crawl, open source code github, wikipedia, Books (Project Gutenberg), Multilingual corpora (OSCAR, C4, The Pile)...
# Technical Q&A and forum (Reddit, StackExchange)
qa_pipeline = pipeline("text-generation", model="NousResearch/Nous-Hermes-2-Mistral-7B-DPO") #mistralai/Mistral-7B-Instruct-v0.1

def retrieve_and_answer(query, chunks, index, embeddings, top_k=3):
    query_vec = model.encode([query]) 
    distances, indices = index.search(query_vec, top_k)
    retrieved_texts = [chunks[i] for i in indices[0]]
    
    context = "\n--------------------------------------------------\n".join(retrieved_texts)
    prompt = f"Answer this question based on the context below:\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
    
    response = qa_pipeline(prompt, max_new_tokens=200, do_sample=True)[0]['generated_text']
    return response

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Device set to use cpu


# Run Full Pipeline

In [8]:
pdf_path = "/kaggle/input/dataset/WWII_Key_Events_2pages.pdf"
text = extract_text_from_pdf_by_sentences(pdf_path)
chunks = chunk_text(text)
embeddings = embed_chunks(chunks)
index = build_faiss_index(np.array(embeddings))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
import time
question = "What caused the United States to enter World War II?"
initial = time.time()
answer = retrieve_and_answer(question, chunks, index, embeddings)
final = time.time()
print("time in minutes:", (final-initial) /  60)
print(answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

time in minutes: 3.0092994372049966
Answer this question based on the context below:

Context:
Operation Barbarossa (1941)
In June 1941, Nazi Germany launched Operation Barbarossa, a massive invasion of the Soviet
Union. This marked a major escalation of the war and opened the Eastern Front, which became the
largest and bloodiest theater of conflict in World War II. Despite early successes, German forces
were eventually halted outside Moscow and faced fierce Soviet resistance, especially in Stalingrad. Pearl Harbor (1941)
On December 7, 1941, the Japanese navy launched a surprise attack on the U.S. naval base at
Pearl Harbor in Hawaii. The attack killed over 2,400 Americans and led the United States to declare
war on Japan the following day. This event brought the United States into the global conflict, which
shifted the balance of power among the Allied and Axis forces. Battle of Stalingrad (1942-1943)
One of the turning points of the war, the Battle of Stalingrad saw Soviet forces de

# Activity

1. Ask 5 questions and evaluate model answers qualitatively. Add brief comments.

2. Calculate BLEU score for the answers. Comment on the results.

3. Upload your own PDF, ask 5 new questions, and comment on the new answers.

4. Calculate BLEU score again and compare it with the previous results.